In [1]:
import sys
sys.version

'3.7.7 (default, Mar 10 2020, 15:43:33) \n[Clang 11.0.0 (clang-1100.0.33.17)]'

In [2]:
import re
import os
import unicodedata
import pandas as pd
import numpy as np
import pickle
import requests
import bs4 as bs
from lxml import html
from tqdm import tqdm
import time

In [3]:
os.path

<module 'posixpath' from '/usr/local/Cellar/python/3.7.7/Frameworks/Python.framework/Versions/3.7/lib/python3.7/posixpath.py'>

# Get companies and cik from wikipedia

## use BeautifulSoup

In [4]:
resp = requests.get('http://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
soup = bs.BeautifulSoup(resp.text, 'lxml')
table = soup.find('table', {'class': 'wikitable sortable'})
tickers = []
CIKs=[]
for row in table.findAll('tr')[1:]:
    ticker = row.findAll('td')[1].text
    tickers.append(ticker)
    CIK=row.findAll('td')[7].text
    CIKs.append(CIK)
    
data={'Company':tickers, 'CIK':CIKs}
company_data=pd.DataFrame.from_dict(data)
company_data=company_data.astype(object)
company_data.to_csv('AllTickers.csv',encoding='utf-8',index=False)

In [5]:
df_trickers=pd.read_csv('AllTickers.csv')

In [6]:
#df_trickers=df_trickers[0:2]

In [7]:
df_trickers

,Company,CIK
0,3M Company,66740
1,Abbott Laboratories,1800
2,AbbVie Inc.,1551152
3,ABIOMED Inc,815094
4,Accenture plc,1467373
...,...,...
500,Yum! Brands Inc,1041061
501,Zebra Technologies,877212
502,Zimmer Biomet Holdings,1136869
503,Zions Bancorp,109380


# Log file control

In [8]:
def SaveLogFile(log_file_name, text):
    
    with open(log_file_name, "a") as log_file:
        log_file.write(text)

    return

def CloseLogFile(log_file_name):
    
    with open(log_file_name, 'a') as log_file:
        log_file.close()

    return

# The path will store 10-K and 10-Q

In [9]:
s10k_path = "/Users/buckycheng/Jupyter/Huge_Stock/10K"
s10q_path = "/Users/buckycheng/Jupyter/Huge_Stock/10Q"

# Get 10-K File
## 01/01/2008 to now

In [12]:
def Get10K(browse_url_base, filing_url_base, doc_url_base, cik, log_file_name):
    
    requestcount=0
    if not os.path.exists(cik):
        os.makedirs(cik)
    
    os.chdir(cik)
    print('Now CIK ', cik)
    
    SaveLogFile(log_file_name, 'Now CIK: '+cik)
            
    res = requests.get(browse_url_base % cik)
    requestcount+=1
    if(requestcount==10):
        time.sleep(1)
        requestcount=0
    #request fail
    if res.status_code != 200:
        text = "\nFailed at step 1.\nCIK error " + str(res.status_code) + \
               "\nFailed URL: " + (browse_url_base % cik) + '\n'
        
        SaveLogFile(log_file_name, text)
            
        return

    SaveLogFile(log_file_name,"\nSuccessful Web UrL\n")
        
    #using BeautifulSoup clean html
    soup = bs.BeautifulSoup(res.text, "lxml")

 
    html_tables = soup.find_all('table')
    
    if len(html_tables)<3:
        os.chdir('..')
        return
    
    
    sec_filing_tbl = pd.read_html(str(html_tables[2]), header=0)[0]
    sec_filing_tbl['Filings'] = [str(x) for x in sec_filing_tbl['Filings']]

    sec_filing_tbl = sec_filing_tbl[(sec_filing_tbl['Filings'] == '10-K') | (sec_filing_tbl['Filings'] == '10-K405')]
    sec_filing_tbl = sec_filing_tbl[(sec_filing_tbl['Filing Date'] >= '2008-01-01')]

    if len(sec_filing_tbl)==0:
        os.chdir('..')
        return
    
    sec_filing_tbl['Acc_No'] = [x.replace('\xa0',' ')
                               .split('Acc-no: ')[1]
                               .split(' ')[0] for x in sec_filing_tbl['Description']]

   
    SaveLogFile(log_file_name, "Getting all documents for this CIK\n")
    
    total_files_to_scrape = len(sec_filing_tbl)
    files_scraped = 0
    
    
    for index, row in sec_filing_tbl.iterrows():
        
       
        acc_no = str(row['Acc_No'])
        
        
        date = str(row['Filing Date'])
        if (os.path.exists(cik + '_' + date + '.html')) or os.path.exists(cik + '_' + date + '.txt'):
            SaveLogFile(log_file_name, "The file for date: " + date + " exists, acc no.: " + acc_no + "\n")
            files_scraped+=1
            continue
            
   
        docs_page = requests.get(filing_url_base % (cik, acc_no))
        requestcount+=1
        if(requestcount==10):
            time.sleep(1)
            requestcount=0
        
        
        #if request fail,store in log and go on
        if docs_page.status_code != 200:
            text = "Request failed with error code " + str(docs_page.status_code) + \
                   "\nFailed URL: " + (filing_url_base % (cik, acc_no)) + '\n'
            SaveLogFile(log_file_name, text)
            continue
            
        SaveLogFile(log_file_name, "Got acc no. " + acc_no + "\n")
       
        docs_page_soup = bs.BeautifulSoup(docs_page.text, 'lxml')
        docs_html_tables = docs_page_soup.find_all('table')
        if len(docs_html_tables)==0:
            continue
        docs_table = pd.read_html(str(docs_html_tables[0]), header=0)[0]
        docs_table['Type'] = [str(x) for x in docs_table['Type']]
        
 
        docs_table = docs_table[(docs_table['Type'] == '10-K') | (docs_table['Type'] == '10-K405')]
        
        if len(docs_table)==0:
            continue
        elif len(docs_table)>0:
            docs_table = docs_table.iloc[0]
        
        docname = docs_table['Document']
        
         # check nan 
        if str(docname) == 'nan':
            text = 'File with CIK: %s and Acc_No: %s is unavailable' % (cik, acc_no) + '\n'
            SaveLogFile(log_file_name, text)
            continue       
        
        file = requests.get(doc_url_base % (cik, acc_no.replace('-', ''), docname.replace(' iXBRL','')))
        requestcount+=1
        if(requestcount==10):
            time.sleep(1)
            requestcount=0
            
       
        if file.status_code != 200:
            os.chdir('..')
            text = "Request failed with error code " + str(file.status_code) + \
                   "\nFailed URL: " + (doc_url_base % (cik, acc_no.replace('-', ''), docname)) + '\n'
            SaveLogFile(log_file_name, text)
            os.chdir(cik)
            continue
        
        SaveLogFile(log_file_name, "Got 10K for acc no. " + acc_no + "\n")
        files_scraped+=1
        
        
        if '.txt' in docname:
            filename = cik + '_' + date + '.txt'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        else:
            filename = cik + '_' + date + '.html'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
            
    if(total_files_to_scrape!=files_scraped):
        print("Some files failed to scrape\n")
        text="Some files failed to scrape\n" + str(total_files_to_scrape) + "!="+ str(files_scraped) + "\n"
        SaveLogFile(log_file_name, text)
        
    SaveLogFile(log_file_name, "saving log files=================\n")
    CloseLogFile(log_file_name)
    os.chdir('..')

In [14]:

# the URL of U.S. Sucurities and Exchange Commission
browse_url_10k = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=%s&type=10-K'
filing_url_10k = 'http://www.sec.gov/Archives/edgar/data/%s/%s-index.html'
doc_url_10k = 'http://www.sec.gov/Archives/edgar/data/%s/%s/%s'

# moving the directory, where scraped 10-K filings are
os.chdir(s10k_path)
log_file = 'log.txt'

for cik in tqdm(df_trickers['CIK']):
    Get10K(browse_url_base=browse_url_10k, 
          filing_url_base=filing_url_10k, 
          doc_url_base=doc_url_10k, 
          cik=str(cik),
          log_file_name=log_file)

  0%|          | 0/505 [00:00<?, ?it/s]

Now CIK  66740


  0%|          | 1/505 [00:01<09:34,  1.14s/it]

Now CIK  1800


  0%|          | 2/505 [00:02<09:32,  1.14s/it]

...

100%|█████████▉| 503/505 [54:17<00:32, 16.17s/it]

Now CIK  109380


100%|█████████▉| 504/505 [54:36<00:17, 17.13s/it]

Now CIK  1555280


100%|██████████| 505/505 [54:47<00:00,  6.51s/it]


# Get 10-Q File
## 01/01/2008 to now

same as get 10-K File

In [15]:
def Get10Q(browse_url_base, filing_url_base, doc_url_base, cik, log_file_name):
    
    requestcount=0

    if not os.path.exists(cik):
        os.makedirs(cik)
    
    os.chdir(cik)
    print('Now CIK ', cik)
    
    SaveLogFile(log_file_name, 'Now CIK: '+cik)
            
    res = requests.get(browse_url_base % cik)
    requestcount+=1
    if(requestcount==10):
        time.sleep(1)
        requestcount=0
    
     #Request Fail
    if res.status_code != 200:
        text = "\nFailed \n CIK Error " + str(res.status_code) + \
               "\nFailed URL: " + (browse_url_base % cik) + '\n'
        
        SaveLogFile(log_file_name, text)
            
        return

   
    SaveLogFile(log_file_name,"\nSuccessfully Web UrL\n")
        
    soup = bs.BeautifulSoup(res.text, "lxml")

    # fetching all the html tables
    html_tables = soup.find_all('table')
    
    
    if len(html_tables)<3:
        os.chdir('..')
        return
    
    # Parss the Filing Table
    sec_filing_tbl = pd.read_html(str(html_tables[2]), header=0)[0]
    sec_filing_tbl['Filings'] = [str(x) for x in sec_filing_tbl['Filings']]


    sec_filing_tbl = sec_filing_tbl[sec_filing_tbl['Filings'] == '10-Q']
    sec_filing_tbl = sec_filing_tbl[(sec_filing_tbl['Filing Date'] >= '2008-01-01')]
    
    if len(sec_filing_tbl)==0:
        os.chdir('..')
        return
    
  
    sec_filing_tbl['Acc_No'] = [x.replace('\xa0',' ')
                               .split('Acc-no: ')[1]
                               .split(' ')[0] for x in sec_filing_tbl['Description']]

   
    SaveLogFile(log_file_name, "Getting all documents for this CIK\n")
    
    total_files_to_scrape = len(sec_filing_tbl)
    files_scraped = 0
    
    for index, row in sec_filing_tbl.iterrows():
        
     
        acc_no = str(row['Acc_No'])
        
        # check if file already exist
        date = str(row['Filing Date'])
        if (os.path.exists(cik + '_' + date + '.html')) or os.path.exists(cik + '_' + date + '.txt'):
            SaveLogFile(log_file_name, "The file for date: " + date + " exists, acc no.: " + acc_no + "\n")
            files_scraped+=1
            continue
            
      
        docs_page = requests.get(filing_url_base % (cik, acc_no))
        requestcount+=1
        if(requestcount==10):
            time.sleep(1)
            requestcount=0
        
        #In case of request failure,log it and jump to the next filing 
        if docs_page.status_code != 200:
            text = "Request failed with error code " + str(docs_page.status_code) + \
                   "\nFailed URL: " + (filing_url_base % (cik, acc_no)) + '\n'
            SaveLogFile(log_file_name, text)
            continue
            
        SaveLogFile(log_file_name, "Got acc no. " + acc_no + "\n")

        docs_page_soup = bs.BeautifulSoup(docs_page.text, 'lxml')
        docs_html_tables = docs_page_soup.find_all('table')
        
        if len(docs_html_tables)==0:
            continue
            
        docs_table = pd.read_html(str(docs_html_tables[0]), header=0)[0]
        docs_table['Type'] = [str(x) for x in docs_table['Type']]
        
        docs_table = docs_table[docs_table['Type'] == '10-Q']
        
        if len(docs_table)==0:
            continue
            
        elif len(docs_table)>0:
            docs_table = docs_table.iloc[0]
        
        docname = docs_table['Document']
    
        # check nan values
        if str(docname) == 'nan':
            text = 'File with CIK: %s and Acc_No: %s is unavailable' % (cik, acc_no) + '\n'
            SaveLogFile(log_file_name, text)
            continue       
        
       
        file = requests.get(doc_url_base % (cik, acc_no.replace('-', ''), docname.replace(' iXBRL','')))
        requestcount+=1
        if(requestcount==10):
            time.sleep(1)
            requestcount=0
        
        #Request Failure,
        if file.status_code != 200:
            os.chdir('..')
            text = "Request failed " + str(file.status_code) + \
                   "\nFailed URL: " + (doc_url_base % (cik, acc_no.replace('-', ''), docname)) + '\n'
            SaveLogFile(log_file_name, text)
            os.chdir(cik)
            continue
        
      
        SaveLogFile(log_file_name, "Got 10Q for acc no. " + acc_no + "\n")
        files_scraped+=1
        
         #save file 
        if '.txt' in docname:
            filename = cik + '_' + date + '.txt'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
        else:
            filename = cik + '_' + date + '.html'
            html_file = open(filename, 'a')
            html_file.write(file.text)
            html_file.close()
    
    if(total_files_to_scrape!=files_scraped):
        print("Have files failed to scrape\n")
        text="Have files failed to scrape\n" + str(total_files_to_scrape) + "!="+ str(files_scraped) + "\n"
        SaveLogFile(log_file_name, text)
        
    SaveLogFile(log_file_name, "saving log files==========================\n")
    CloseLogFile(log_file_name)
    os.chdir('..')
        
    return
    
    

In [16]:
# the URL of U.S. Sucurities and Exchange Commission
browse_url_10q = 'https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany&CIK=%s&type=10-Q&count=1000'
filing_url_10q = 'http://www.sec.gov/Archives/edgar/data/%s/%s-index.html'
doc_url_10q = 'http://www.sec.gov/Archives/edgar/data/%s/%s/%s'

# moving the directory, where scraped 10-K filings are
os.chdir(s10q_path)
log_file = 'log.txt'

for cik in tqdm(df_trickers['CIK']):
    Get10Q(browse_url_base=browse_url_10q, 
          filing_url_base=filing_url_10q, 
          doc_url_base=doc_url_10q, 
          cik=str(cik),
          log_file_name=log_file)

  0%|          | 0/505 [00:00<?, ?it/s]

Now CIK  66740


  0%|          | 1/505 [00:50<7:02:15, 50.27s/it]

Now CIK  1800


  0%|          | 2/505 [01:36<6:52:14, 49.17s/it]

...

100%|█████████▉| 503/505 [6:21:30<01:34, 47.02s/it]

Now CIK  109380


100%|█████████▉| 504/505 [6:22:19<00:47, 47.83s/it]

Now CIK  1555280


100%|██████████| 505/505 [6:22:49<00:00, 45.48s/it]


# Prasing HTML file to Text file

## delete the numeric tables

In [17]:
def DeleteNumTables(BSoup):
    
    def GetNumAmount(tablecontent):
        if len(tablecontent)>0.0:
            numbers = sum([char.isdigit() for char in tablecontent])
            length = len(tablecontent)
            return numbers/length
        else:
            return 1
        
    [x.extract() for x in BSoup.find_all('table') if GetNumAmount(x.get_text())>0.15]
    
    return BSoup

## delete HTML tags 

In [18]:
def DeleteHTMLTags(BSoup):
    
    text = BSoup.get_text()
    text = text.replace('\n', ' ')
    text = unicodedata.normalize('NFKD', text)
    
    return text

## HTML content to Text content

In [19]:
def ParseHTMLToText(cik):
    try: 
        os.chdir(cik)
  
    except FileNotFoundError:
        print("No this CIK %s" % cik)
        return
        
    print("Now CIK %s" % cik)
    #check weather flie parsed
    parsed = False 
    
    # Creating a new directory for store text file
    if os.path.exists('rawtext'):
        os.makedirs('clean_data')
        os.rename("rawtext", "clean_data")
    elif not os.path.exists('clean_data'):
        os.makedirs('clean_data')
      
    file_list = [fname for fname in os.listdir() if not (fname.startswith('.') | os.path.isdir(fname) | fname.startswith('log'))]
 
    for filename in file_list:
        new_filename = filename.replace('.html', '.txt')
        text_file_list = os.listdir('clean_data')
        if new_filename in text_file_list:
            continue
               
        #Cleaning the file by removing numerical tables, tags etc.
        with open(filename, 'r') as file:
            parsed = True
            BSoup = bs.BeautifulSoup(file.read(), "lxml")
            BSoup = DeleteNumTables(BSoup)
            text = DeleteHTMLTags(BSoup)
            hex_handle = re.sub(r'[^\x00-\x7f]',' ', text)
            with open('clean_data/'+new_filename, 'w') as newfile:
                newfile.write(hex_handle)
    # Once all the files parsed, log them
    if parsed==True:
        print("Parse CIK finish", cik)
    
    os.chdir('..')
    return

# Parse the 10-K file

In [20]:
os.chdir(s10k_path)
for cik in tqdm(df_trickers['CIK']):
    ParseHTMLToText(str(cik))

  0%|          | 0/505 [00:00<?, ?it/s]

Now CIK 66740


  0%|          | 1/505 [00:34<4:51:34, 34.71s/it]

Parse CIK finish 66740
Now CIK 1800


  0%|          | 2/505 [00:53<4:11:38, 30.02s/it]

...

Parse CIK finish 1136869
Now CIK 109380


100%|█████████▉| 504/505 [3:03:38<00:24, 25.00s/it]

Parse CIK finish 109380
Now CIK 1555280


100%|██████████| 505/505 [3:03:50<00:00, 21.84s/it]

Parse CIK finish 1555280


# Parse the 10-Q file

In [21]:
os.chdir(s10q_path)
for cik in tqdm(df_trickers['CIK']):
    ParseHTMLToText(str(cik))

  0%|          | 0/505 [00:00<?, ?it/s]

Now CIK 66740


  0%|          | 1/505 [00:53<7:33:13, 53.96s/it]

Parse CIK finish 66740
Now CIK 1800


  0%|          | 2/505 [01:13<6:06:40, 43.74s/it]

...

Parse CIK finish 1136869
Now CIK 109380


100%|█████████▉| 504/505 [4:20:57<00:35, 36.00s/it]

Parse CIK finish 109380
Now CIK 1555280


100%|██████████| 505/505 [4:21:16<00:00, 31.04s/it]

Parse CIK finish 1555280
